In [2]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score,r2_score,classification_report,roc_auc_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,RandomForestRegressor
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV,StratifiedKFold
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from joblib import Parallel,delayed

def svar(X):
    n = float(len(X))
    svar=(sum([(x-np.mean(X))**2 for x in X]) / n)* n/(n-1.)
    return svar

def CronbachAlpha(itemscores):
    itemvars = [svar(item) for item in itemscores]
    tscores = [0] * len(itemscores[0])
    for item in itemscores:
        for i in range(len(item)):
            tscores[i]+= item[i]
    nitems = len(itemscores)
    Calpha=nitems/(nitems-1.) * (1-sum(itemvars)/ svar(tscores))
    return Calpha

from scipy.stats import iqr,skew,kurtosis,pearsonr
from scipy.stats import variation,moment

def get_features1(x):
    temp = []
    temp.append(x[-1])
    temp.append(x[0])
    temp.append(x[len(x)//2])
    temp.append(iqr(x))
    temp.append(skew(x))
    temp.append(kurtosis(x))
    temp.append(variation(x))
    temp.append(iqr(x[:len(x)//2]))
    temp.append(skew(x[:len(x)//2]))
    temp.append(kurtosis(x[:len(x)//2]))
    temp.append(variation(x[:len(x)//2]))
    temp.append(iqr(x[len(x)//2:]))
    temp.append(skew(x[len(x)//2:]))
    temp.append(kurtosis(x[len(x)//2:]))
    temp.append(variation(x[len(x)//2:]))
    return np.array(temp)

# def get_features()

def get_features(temp,user):
    if temp.shape[0]<10:
        return -1,-1,-1,-1,-1
    if temp['time'].values[-1]-temp['time'].values[0]>1000:
        return -1,-1,-1,-1,-1
    if np.isnan(temp['score'].values[-1]):
        return -1,-1,-1,-1,-1
    a = temp['stress_likelihood'].values
    b = temp['all_scores'].values[-1]
    c = user
    hour = datetime.fromtimestamp(temp['time'].values[0]).hour
    if hour<8:
        d = 0
    elif hour>8 and hour < 16:
        d = 1
    else:
        d = 2
    e = user+temp['day'].values[-1]
    return a,b,c,d,e

def get_features2(temp,user):
    if temp.shape[0]<10:
        return -1,-1,-1,-1,-1
    if temp['time'].values[-1]-temp['time'].values[0]>1000:
        return -1,-1,-1,-1,-1
    g = temp['score'].values
    if len(g[~np.isnan(g)])>0:
        return -1,-1,-1,-1,-1
#     if np.isnan(temp['score'].values[-1]):
#         return -1,-1,-1,-1,-1
    a = temp['stress_likelihood'].values
    b = temp['all_scores'].values[-1]
    c = user
    hour = datetime.fromtimestamp(temp['time'].values[0]).hour
    if hour<8:
        d = 0
    elif hour>8 and hour < 16:
        d = 1
    else:
        d = 2
    e = user+temp['day'].values[-1]
    return a,b,c,d,e

def get_user_data(user_data,user):
    X_date = []
    X = []
    y = []
    groups = []
    days = []
    if len(np.unique(user_data.dropna()['day'].values))<4:
        return [],[],[],[],[]
    user_data = user_data.sort_values('localtime').reset_index(drop=True)
    count = 0
    columns=['window', 'localtime', 'day', 'stress_likelihood', 'imputed',
       'likelihood_mean', 'all_scores', 'score', 'label', 'version', 'user',
       'time']
    count = 0
    data = Parallel(n_jobs=20,verbose=1)(delayed(get_features)(user_data.loc[i:i+10],user) for i,row in user_data.iterrows()
                                        if not np.isnan(user_data.loc[i:i+10]['score'].values[-1]))
    data1 = Parallel(n_jobs=20,verbose=1)(delayed(get_features2)(user_data.loc[i:i+10],user) for i,row in user_data.iterrows()
                                        if i%10==0)
    
    data = data+data1
    for a in data:
        if not isinstance(a[2], str):
            continue
        X.append(a[0])
        y.append(a[1])
        groups.append(a[2])
        X_date.append(a[3])
        days.append(a[4])
    print(len(X))
    return X,y,groups,X_date,days


def get_data(all_stress):
    X_date,X,y,groups,days = [],[],[],[],[]
    final = [get_user_data(all_stress[all_stress.user.isin([user])],user) 
                                          for user in np.unique(all_stress['user'].values)]
    for a in final:
        X.extend(a[0])
        y.extend(a[1])
        groups.extend(a[2])
        X_date.extend(a[3])
        days.extend(a[4])
    return X,y,groups,X_date,days

In [ ]:
import warnings
import pickle
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler,OneHotEncoder
all_stress_left = pickle.load(open('../data/stress_ema_md2k_aa_rice_left_ppg.p','rb'))
# all_stress_left = all_stress_left[all_stress_left.imputed.isin([0])]
all_stress_right = pickle.load(open('../data/stress_ema_md2k_aa_rice_right1_ppg.p','rb'))
# all_stress_right = all_stress_right[all_stress_right.imputed.isin([0])]

In [ ]:
X2,y2,groups2,X_date2,days2 = get_data(all_stress_left)
X3,y3,groups3,X_date3,days3 = get_data(all_stress_right)
Xf,yf,groupsf,X_datef,daysf = X2+X3,y2+y3,groups2+groups3,X_date2+X_date3,days2+days3

In [ ]:
pickle.dump([Xf,yf,groupsf,X_datef,daysf],open('../data/semi_supervised_no_imputation.p','wb'))

In [3]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler,OneHotEncoder
Xf,yf,groupsf,X_datef,daysf = pickle.load(open('../data/semi_supervised_no_imputation.p','rb'))
X,y1,groups1,X_date_final = np.array(Xf),np.array(yf),np.array(groupsf),OneHotEncoder().fit_transform(np.array(X_datef).reshape(-1,1))
Xfeatures = np.array(Parallel(n_jobs=30,verbose=1)(delayed(get_features1)(x) for x in Xf))
print(Xfeatures.shape)

/home/azim/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 166 tasks      | elapsed:    0.9s
[Parallel(n_jobs=30)]: Done 2880 tasks      | elapsed:    1.5s
[Parallel(n_jobs=30)]: Done 24320 tasks      | elapsed:    3.7s
[Parallel(n_jobs=30)]: Done 53120 tasks      | elapsed:    6.7s
[Parallel(n_jobs=30)]: Done 88320 tasks      | elapsed:   10.3s
[Parallel(n_jobs=30

(174735, 15)


[Parallel(n_jobs=30)]: Done 174735 out of 174735 | elapsed:   19.1s finished


In [4]:
from sklearn.model_selection import train_test_split,ParameterGrid
from frameworks.SelfLearning import SelfLearningModel
from frameworks.CPLELearning import CPLELearningModel
all_y = []
all_y_pred = []
all_groups = []
def get_labels(y):
    temp = np.array([np.mean(a) for a in y])
    temp_mean = np.mean(temp)
    index2 = np.where(temp>temp_mean)[0]
    index1 = np.where(temp<=temp_mean)[0]
    temp[index2] = 1
    temp[index1] = 0
    labels = temp
    labels = np.int64(np.array(labels))
    return labels
def get_model_performance(X_train,y_train,X_test,y_test,params):
    clf = LogisticRegression(**params)
#     clf = SelfLearningModel(clf)
    clf = CPLELearningModel(clf)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test[y_test>-1])
    y_test = y_test[y_test>-1]
    print(np.array([f1_score(y_test,y_pred),roc_auc_score(y_test,y_pred)]))
    return np.array([roc_auc_score(y_test,y_pred),
                     precision_score(y_test,y_pred),
                     recall_score(y_test,y_pred),
                     f1_score(y_test,y_pred)])
    
all_results = []
for user in np.unique(groups1):
    index = np.where(groups1==user)[0]
    X1,y,groups = Xfeatures[index],y1[index],groups1[index]
    index_not_nan = y!=None 
    X_nn,y_nn,groups_nn = X1[index_not_nan],y[index_not_nan],groups[index_not_nan]
    X_n,y_n,groups_n = X1[~index_not_nan],y[~index_not_nan],groups[~index_not_nan]
    y_nn = np.array(get_labels(y_nn))
    if len(np.unique(y_nn))<2:
        continue
    if len(y_nn[y_nn==1])<3:
        continue
    if len(y_nn[y_nn==0])<3:
        continue
    y_n = [-1]*len(y_n)
    X1 = np.concatenate([X_nn,X_n],axis=0)
    labels = np.array(list(y_nn)+y_n)
    print(X1.shape,X_n.shape,X_nn.shape,labels.shape)
    X_train, X_test, y_train, y_test = train_test_split(X1, labels, test_size=0.4, random_state=4,stratify=labels)
    paramGrid = ParameterGrid({
            'C': np.logspace(-3,3,10),
            'class_weight':[{0:1,1:4/2},{0:1,1:3/2},{0:1,1:5/2},{0:1,1:1}]
    })
    result = np.array(Parallel(n_jobs=30,verbose=1)(delayed(get_model_performance)(X_train,y_train,X_test,y_test,params) for params in paramGrid))
    result = result[result[:,0]>=.5]
    all_results.append(result[np.argmax(result[:,-1])])
    print(result[np.argmax(result[:,1])])


(1579, 15) (1559, 15) (20, 15) (1579,)


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done  22 out of  40 | elapsed:    2.0s remaining:    1.6s


[0.5 0.  0.  0. ]
(2950, 15) (2883, 15) (67, 15) (2950,)


[Parallel(n_jobs=30)]: Done  40 out of  40 | elapsed:    2.7s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done  22 out of  40 | elapsed:    2.4s remaining:    1.9s


[0.5 0.  0.  0. ]
(2163, 15) (2104, 15) (59, 15) (2163,)


...

[0.5 0.  0.  0. ]
(2281, 15) (2266, 15) (15, 15) (2281,)


[Parallel(n_jobs=30)]: Done  40 out of  40 | elapsed:    4.4s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done  22 out of  40 | elapsed:    1.9s remaining:    1.6s


[0.5 0.  0.  0. ]
(3289, 15) (3239, 15) (50, 15) (3289,)


[Parallel(n_jobs=30)]: Done  40 out of  40 | elapsed:    3.0s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done  22 out of  40 | elapsed:    2.5s remaining:    2.1s


KeyboardInterrupt: 

In [ ]:
all_results = np.array(all_results)

plt.figure(figsize=(9,6))
plt.boxplot(all_results)
plt.show()

In [ ]:
all_results = all_results[all_results[:,0].argsort()]


In [ ]:
plt.figure()
roc = all_results[:,0]
precision = all_results[:,1]
recall = all_results[:,2]
f1 = all_results[:,3]
x = np.array([i*2 for i in range(len(roc))])
plt.figure(figsize=(16,8))
plt.bar(x,roc,1)
plt.show()
plt.figure(figsize=(16,8))
plt.bar(x,precision,1)
plt.show()
plt.figure(figsize=(16,8))
plt.bar(x,recall,1)
plt.show()
plt.figure(figsize=(16,8))
plt.bar(x,f1,.5)
plt.show()
# plt.show()

In [ ]:
len(roc)

In [ ]:
from frameworks.CPLELearning

In [ ]:
X[0]

In [ ]:
bar_results.shape

In [ ]:
all_results.shape